# Parse development book

### Notebook purpose
This notebook is development space for python parse.ts replacement and upgrade.
It reads specified google sheets and output actants.json file, which can be imported to inkVisitor RethinkDB.py

The aggreated doc for dev: https://docs.google.com/document/d/1ga6R_9TWAQlXE9XqPE_qZ2S8S8aVW2A7n8SuYpSXnoI/edit#heading=h.q9ntf0ofam2u
The holy schema: https://app.diagrams.net/#G1bKvqEKr6JzPWryVg-vYudQy_4KvuzHS9

### JSon schemas for the actants
 * are created from *.ts files through typescript-jschema commandline utility
 * the schemas are trasformed as python Classes through warlock library


### Principles of the parsing operations
 * There are google sheet dataset, which need to be transformed to json format according to inkVisior datamodel
 * inkVisitor holds datamodel in the code, the typescript classes (see /shared/types...), or the holy schema
 * DZ created in input table parsing instructions
   * the first 5 rows contain instructions
   * if the keywords contain ? character, they are ignored (it is work in progress from DZ)

#### Parsing instructions
 * explicit set of keywords
   * discard
     * ...
   * inside
     * the value in the column should be straitforwadly made part of the entity object, the exists update_generic method or update_fieldName mehthods for fields with custom but generally applicable logic
   * propvalue
     * for making so called
          * statement or property statment (old A0093 has relation)
     * it sits in the entity "props" attribute, it has IProp class
   * special
     * fully custom method for the parsing behavior
     *

### Parsing process
 1. Prepare data model entities classes (from typescript inkvisitor classes -> json schema ->  python classes)
 2. Load and wrangle all input data
 3. Process headers of the tables for parsing instructions
 4. Process tables row by row, column by column
 5. Save as json file (which can be imported to the inkVisitor RethinkDB)


# DEV Update May 2024
preparing for "operation reimpoirt"

# DEV Update January 2023
encapsulation of code


# DEV Updates : November
 - TODO updated model for relations (attribute order)
 - TODO synonymic relations parsed to "cloud"

# Dev updates:  October 2
- DONE change event-territories generation,  genre is relation.classification, not metaprop
- TODO prepare for "partial parsing" and fixed guids for "done" tables
- DONE updated users
- DONE change in Manuscript parsing, For each non-empty, non-NA value: check whether the script hasn't already generated an L entity with the same label.
IF NOT: (1) Generate a new L entity with label = value in this col., status = "approved", entity logical type = "definite", label language = value in the next col. (repository_label_language); (2) append to this L entity a RelationType.Classification leading to C2646 "archive or library". (3) Go to No. 4.
IF YES, go to No. 4 and use that L.
(4) Under the O entity representing the row (i.e. the physical manuscript), add a metaprop which will relate this O to this L entity (repository) through the relation: O - (has) - C2645 "repository" - L in this col.




# Dev updates:  October
 - DONE u action a conceptu se neparsuji relations ?
 - DONE revize skriptu vzhledem k september bordelu
 - DONE events parsing
 - DONE 21.09. 2022, R table has received a new parsable column “components_id”. Its notation follows our agreed-upon notation, so it should not pose any problem, but I rather signal this new col. to you. It is for composite Rs, which need to be separated into several identities with different URIs (typically two volumes of an edition)
  - DONE The instruction here and here for generating creation events for Territories and manuscript Objects has changed, since what was the Property C0565 class is now the RelationType Classification. Please take this change into account, tx.
 - DONE territories generation
  - light yellow: metadata of Subterritory capturing the lowest-level document recording the Event (not of Event described by the row). Tomáš H.: (1) Please generate these lowest-level T entities following Adam’s script for parsing Robert’s Sellan coding sheet IDs into a hierarchical T structure and put them under their proper root T through legacy IDs: first element of ID = root T ID, last element = lowest-level Territory whose label follows in the next col., middle elements = folders in between root and lowest child level. E.g. here, in Guglielmites, there is root, i.e. the whole Guglielmites process of Beltramo Salvagno; then parts 1-4; and underneath them, the individual documents. When creating these lowest-level Subterritories, use this attributes: Entity type = T; T label and label_language defined in the two respective cols. which follow; status = approved. Other metadata set as metaprops in the following yellow cols. (2) Generate the middle entities holding the hierarchy, name them by the chunk ID (in this case, this will be four Ts named “1”, “2”, “3”, and “4”), and relate the lowest-level entity to them as their parents, and relate those middle entities to the root T as their parent in turn; attributes of these middle entities (folders 1-4 of the records): label = number [1,2,3,4] depending on the chunk; legacy id = T107-[chunk number], label language = Latin, status = approved. (3) Relate the lowest-level entity to the Event described by the row by the following Metaproperty: T107-1-01 - [has] - C2286 represented event - E0001 (middle-level entities will have no such relation now, only lowest-level documents).


# Dev updates:  September
 - adding new users

# Dev updates:  August
 - new instruction with proptype_2nd in column
    - before we got the type fixed and had just instruction propvalue_2nd (used in territories for manuscript witness)
 - new instruction "hook-inside" for insides in newly created entities (see in gugli locations special:modern_name)
 - changes :  some properties 'id' changed to 'specificId'
   - IProp {... 'value':{'entityId' and 'type':{'entityId'
   - IStatement {  'data':{ 'territory' {'territoryId'
   - ITerritory { 'data': {'parent': {'territoryId': ""
- bigger TODO relations

# Dev updates: April

## Concepts
  * DONE wordnet_synset_id
    * wordnet_id bounded to resource object R0067

## Texts
 * DONE special_creation_event_id
   * instructions:
     * Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to any of those Es the metaprop "(has) - C0565 "class" - C2642 "creation" (to instantiate the event to its event type = event class).
   * creation_event_label in standalone field
     * this will trigger the operation
   * lot of proptype and propvalue with non-standard schema: new branch of interpreting

## Manuscripts
 * special_repository_label
   * i:
     * For each non-empty, non-NA, non-NS row: (1) generate L entity with label = value in this col., status = "approved", entity logical type = "definite", label language = value in the next col. (repository_label_language); (2) append to this L entity a metaprop (has) - C0565 "class" - C2646 "archive or library", and (3) under the O entity representing the row (i.e. the physical manuscript), add a metaprop which will relate this O to this L entity (repository) through the relation: O - (has) - C2645 "repository" - L in this col.
   * repository_label_language
     * i:
       * Use this value as label language value of the repository L entity.
 * DONE creation_event_id
   * creation_event_label
   * i:
     * Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to any of those Es the metaprop "(has) - C0565 "class" - C2642 "creation" (to instantiate the event to its event type = event class).
 * reproduction_online_url
   * instructions:
     * If non-empty, non-NA, (1) generate an R entity with label "Reproduction of " + label of the MS (i.e. value in the B column, status = "approved", label-language = "English", url = the URL sitting under the hyperlink value in this cell, and (2) add metaprop to the O entity represented by this row: O - (has) - C1199 "digital reproduction" - the R entity here generated.

### Input variables

In [1]:
input_tables = ["texts", "manuscripts", "resources", "actions" , "concepts"]

#                   sheet_name,  gdrive code, header_in_row, columns which need suffix based on a case name
input_sheets = {
    "texts" : ("Texts","13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5), #https://docs.google.com/spreadsheets/d/13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE/edit#gid=2056508047
    "manuscripts" : ("Manuscripts", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "resources" : ("Resources", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "actions" :  ("Statements","1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ", 5), # https://docs.google.com/spreadsheets/d/1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ/edit#gid=0
    "concepts" : ("Concepts","1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4", 5), # https://docs.google.com/spreadsheets/d/1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4/edit#gid=0,
    "R0006_persons" : ("Persons","1UtgKvjr91egdgOQMP8WxsRxgGDT9FC92qBG3sKBiYvk", 5, ['id','residence_id','social_relation1_value_id','social_relation2_value_id','social_relation3_value_id']), # GUGLIELMITES Persons, https://docs.google.com/spreadsheets/d/1UtgKvjr91egdgOQMP8WxsRxgGDT9FC92qBG3sKBiYvk/edit#gid=0,
    "R0007_locations" : ("Locations","1jImuYCTu-QrZc7oCT1Vz01SLu5sxYRo4xzJzlF2J8Bs", 5, ['id','locationrelation1_value_id','superordinate_location_id']) ,# GULIELMITES Locations, https://docs.google.com/spreadsheets/d/1jImuYCTu-QrZc7oCT1Vz01SLu5sxYRo4xzJzlF2J8Bs/edit#gid=0,
    "R0008_events" : ("Events","1mLevawnoZVLQv6qINaSfiyi1NNlfJzTZevinRSSQVPo", 5, ['id','spatial_ref_value_id','participant_id','inquisitor_id','notary_id','witness_assessor_id','accuser_id','accused_id','mentioned_id','denied_id','marked_as_incriminated_id','temporal_ref_1_value_id','temporal_ref_2_value_id']),
    # Guglielmites EVENTS https://docs.google.com/spreadsheets/d/1mLevawnoZVLQv6qINaSfiyi1NNlfJzTZevinRSSQVPo/edit#gid=0,
    "R0075_persons" : ("Persons","1Tyi4DYnHDeTs8tuz9rxN3wyq1LqLWWp_ZPgOimQoxf8", 5, ['id','origin_or_residence_id','residence_id','relations_1_value_id','relations_2_value_id','relations_3_value_id','relations_4_value_id','relations_5_value_id','relations_6_value_id','relations_7_value_id','relations_8_value_id','relations_9_value_id','relations_10_value_id','relations_11_value_id','relations_12_value_id','relations_13_value_id','origin_id','occupation_id','office_id']), # Piedmont Castellario Persons, https://docs.google.com/spreadsheets/u/1/d/1Tyi4DYnHDeTs8tuz9rxN3wyq1LqLWWp_ZPgOimQoxf8/edit#gid=0
    "R0035_locations" : ("Locations","1VwUDnAMJ8mWvL-54K-OnZmguB-rALnJjtGG2IGNBqIU", 5, ['id','locationrelation1_value_id','locationrelation2_value_id','identificate_id','superordinate_location_id']) ,# Piedmont Castellario Locations, https://docs.google.com/spreadsheets/d/1VwUDnAMJ8mWvL-54K-OnZmguB-rALnJjtGG2IGNBqIU/edit#gid=0
    "R0083_events" : ("Events","1M5uhJsbjUCxWP_avYQM0fr1Hm0GXVmYuDvqSoEuUAbs", 5, ['id','participant_id','spatial_ref_value_id','inquisitor_id','notary_id','witness_assessor_id','accuser_id','accused_id','mentioned_id','denied_id','temporal_ref_1_value_id','temporal_ref_2_value_id']), 
    # Castellario Incriminations https://docs.google.com/spreadsheets/u/1/d/1M5uhJsbjUCxWP_avYQM0fr1Hm0GXVmYuDvqSoEuUAbs/edit#gid=0
     "R0105_groups" : ("Groups","1IqislE4C0GM45Wvlg9JW2QmzqvR7LgjA5TZTkM143U4", 5, ['id','has_member_id']) ,# Piedmont Castellario Groups, https://docs.google.com/spreadsheets/d/1IqislE4C0GM45Wvlg9JW2QmzqvR7LgjA5TZTkM143U4/edit#gid=0


    "R0030_persons" : ("Persons","1W8YTiKBZZOi_682guwH7N9aEX_qINi9D3qBM8u96Fhk", 5, ['id','toponymic_surname_id','residence_id', 'origin_id','social_relation1_value_id','social_relation2_value_id','social_relation3_value_id','social_relation4_value_id','social_relation5_value_id']), # Bologna Persons, https://docs.google.com/spreadsheets/d/1W8YTiKBZZOi_682guwH7N9aEX_qINi9D3qBM8u96Fhk/edit#gid=0
    "R0087_groups" : ("Groups","1WuTIsyBaC7aD_o8xtS3VcZ8_lmA5T-SABKPIkHyWsPw", 5, ['id']), # Bologna Groups, https://docs.google.com/spreadsheets/d/1WuTIsyBaC7aD_o8xtS3VcZ8_lmA5T-SABKPIkHyWsPw/edit#gid=0
    "R0031_locations" : ("Locations","1AvGxtm3fSg9sl8CrjKiLaLLwTFaerSiz3wBUUjl2Il8", 5, ['id','superordinate_location_id','province_modern_id','locationrelation1_value_id']) ,# Bologna Locations, https://docs.google.com/spreadsheets/d/1AvGxtm3fSg9sl8CrjKiLaLLwTFaerSiz3wBUUjl2Il8
    "R0082_events" : ("Events","1qYTMaY-DwI2XiFTLUL9tiGC7ah9Lkm0YeKWXw6V0yog", 5, ['id','inquisitor_id','notary_id','witness_assessor_id','spatial_ref_value_id','accuser_id','accused_id','mentioned_id','denied_id']), # Bologna Trial Events, https://docs.google.com/spreadsheets/d/1qYTMaY-DwI2XiFTLUL9tiGC7ah9Lkm0YeKWXw6V0yog/edit#gid=0  # 05/2024 deleting participant_id, David renamed it to OLD_ ...

    "R0039_persons" : ("Persons","1jV9RBSZHSsKiMpJIrB58YdsaPxjBe3RxNLpR1ODJyWU", 5, ['id','residence_id','origin_id','origin_or_residence_id','relation1_value_id','relation2_value_id','relation3_value_id','relation4_value_id', 'property_value_id']), # Carcassone HHH Persons, https://docs.google.com/spreadsheets/d/1jV9RBSZHSsKiMpJIrB58YdsaPxjBe3RxNLpR1ODJyWU/edit#gid=0

    # NO GROUPS in Carcassonne

    "R0040_locations" : ("Locations","1FSLbf0IE4zpchuHL1hHOsJEx0jgYGSoHU5RZzt86jLI", 5, ['id','superordinate_location_id','locationrelation1_value_id']) ,# Carcassone HHH  Locations, https://docs.google.com/spreadsheets/d/1FSLbf0IE4zpchuHL1hHOsJEx0jgYGSoHU5RZzt86jLI/edit#gid=0  , # 05/2024, there were bad columns locationrelation1_target_id and locationrelation2_target_id, these does not exist; probably relabeled to locationrelation1_value_id ...?
    "R0079_events" : ("Events","1i0XTsxvhP0vkHXoWEDTNtpuQIg-lWNY7jSOyjMZMHbw", 5, ['id','inquisitor_id','notary_id','witness_assessor_id','spatial_ref_value_id','accuser_id','accused_id','mentioned_id','denied_id']), # Carcassone HHH  Trial Events, https://docs.google.com/spreadsheets/d/1i0XTsxvhP0vkHXoWEDTNtpuQIg-lWNY7jSOyjMZMHbw/edit#gid=0  # 05/2024 missing 'person_before_court_id',

    "R0076_persons" : ("Persons","1yIT3kK9OdslT5FykzTjV1kDPkNVZfhIuayzqTK3hgA4", 5, ['id','relations_1_value_id','relations_2_value_id','origin_or_residence_id','residence_id','origin_id']), # Casasco Persons, https://docs.google.com/spreadsheets/d/1yIT3kK9OdslT5FykzTjV1kDPkNVZfhIuayzqTK3hgA4/edit#gid=0
    "R0074_locations" : ("Locations","1lUKyrZu9r9do9zXjzF7GUhMDgihXVYszwuOA5imuCDc", 5, ['id','locationrelation1_value_id','locationrelation2_value_id','superordinate_location_id','identificate_id']) ,# Casasco  Locations, https://docs.google.com/spreadsheets/d/1lUKyrZu9r9do9zXjzF7GUhMDgihXVYszwuOA5imuCDc/edit#gid=0
    "R0104_events" : ("Events","1XQCBAG__LXnYXmsCiDq-l6iO9cXZTyr4yXV0ISAm-sQ", 5, ['id','participant_id','inquisitor_id','notary_id','witness_id','spatial_ref_value_id','temporal_ref_1_value_id','temporal_ref_2_value_id','accuser_id','accused_id','mentioned_id','denied_id']) # Casasco Trial Events, https://docs.google.com/spreadsheets/d/1XQCBAG__LXnYXmsCiDq-l6iO9cXZTyr4yXV0ISAm-sQ/edit#gid=0
}


# table of tables
# https://docs.google.com/spreadsheets/d/13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE/edit#gid=583750775


# resource IDs
# R0006 Guglielmites Persons
# R0007	Guglielmites Locations
# R0008	Guglielmites Events

# R0035	Piedmont Locations Castellario
# R0075	Persons Castellario
# R0083 Castellario Incriminations
# R0105 Castellario Groups

# RXXXX     Castellario Groups, https://docs.google.com/spreadsheets/d/1IqislE4C0GM45Wvlg9JW2QmzqvR7LgjA5TZTkM143U4/edit#gid=0

# R0087	Bologna Groups
# R0031	Bologna Locations
# R0030	Bologna Persons
# R0082	Bologna Trial Events
# https://docs.google.com/spreadsheets/d/1WuTIsyBaC7aD_o8xtS3VcZ8_lmA5T-SABKPIkHyWsPw/edit#gid=0
# https://docs.google.com/spreadsheets/d/1AvGxtm3fSg9sl8CrjKiLaLLwTFaerSiz3wBUUjl2Il8
# https://docs.google.com/spreadsheets/d/1W8YTiKBZZOi_682guwH7N9aEX_qINi9D3qBM8u96Fhk/edit#gid=0
# https://docs.google.com/spreadsheets/d/1qYTMaY-DwI2XiFTLUL9tiGC7ah9Lkm0YeKWXw6V0yog/edit#gid=0

# R0040	Carcassonne HHH Locations
# R0039	Carcassonne HHH Persons
# R0079	Carcassonne HHH Trial Events
# https://docs.google.com/spreadsheets/d/1FSLbf0IE4zpchuHL1hHOsJEx0jgYGSoHU5RZzt86jLI/edit#gid=0
# https://docs.google.com/spreadsheets/d/1jV9RBSZHSsKiMpJIrB58YdsaPxjBe3RxNLpR1ODJyWU/edit#gid=0
# https://docs.google.com/spreadsheets/d/1i0XTsxvhP0vkHXoWEDTNtpuQIg-lWNY7jSOyjMZMHbw/edit#gid=0

# R0074	Casasco Locations
# R0076	Casasco Persons
# R0104	Casasco Trial Events
# https://docs.google.com/spreadsheets/d/1lUKyrZu9r9do9zXjzF7GUhMDgihXVYszwuOA5imuCDc/edit#gid=0
# https://docs.google.com/spreadsheets/d/1yIT3kK9OdslT5FykzTjV1kDPkNVZfhIuayzqTK3hgA4/edit#gid=0
# https://docs.google.com/spreadsheets/d/1XQCBAG__LXnYXmsCiDq-l6iO9cXZTyr4yXV0ISAm-sQ/edit#gid=0


table_to_entity = {
    "concepts" : "IConcept",
    "resources" : "IResource",
    "texts" : "ITerritory",
    "manuscripts" : "IObject",
    "actions" :  "IAction",
    "R0006_persons" : "IPerson",
    "R0007_locations": "ILocation",
    "R0008_events": "IEvent",
    "R0075_persons" : "IPerson",
    "R0035_locations": "ILocation",
    "R0083_events": "IEvent",
    "R0105_groups": "IGroup",
    "R0030_persons" : "IPerson",
    "R0087_groups" : "IGroup",
    "R0031_locations": "ILocation",
    "R0082_events": "IEvent",
    "R0039_persons" : "IPerson",
    "R0040_locations": "ILocation",
    "R0079_events": "IEvent",
    "R0076_persons" : "IPerson",
    "R0074_locations": "ILocation",
    "R0104_events": "IEvent",
}

root_sheet_url = "https://docs.google.com/spreadsheets/d/"
google_api_dotenv_path = "../env/.env.googleapi"  # contains google api specs for sheet access with Dator

### Libraries

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os, warlock, json
from datetime import datetime
import time
from jsonschema import validate
import dissinetpytools.dator as dator
from dotenv import load_dotenv

import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.filterwarnings("ignore")

import numpy as np
from copy import deepcopy
from shutil import copyfile

import uuid
import dill as pickle
from DataHolder import DataHolder
import inkimplib2


# type hinting
from collections.abc import Sequence, Callable
from typing import List, Dict, Tuple


Creating the DataHolder singleton object


### Initialisation

In [4]:
load_dotenv(google_api_dotenv_path) # fills os.environ['GDRIVE_API_CREDENTIALS']
d = dator.Dator(loglevel=10, print_log_online=True, cache=True, project_name="inkvisitor-import") # expects 'GDRIVE_API_CREDENTIALS' in the global system variables (os.environ)
d.google_authenticate()
logger = d.logger

20 2024-06-01 12:52:37 : Google authentification start
20 2024-06-01 12:52:37 : Google authentification end
20 2024-06-01 12:52:37 : Dator initiation succesfull end


In [34]:
# factory for making entity objects, contains defaults with "prerequisities"

IOF = inkimplib2.InkVisitorJSONObjectFactory("../schemas/","")
IOF.validate_defaults()

2024-06-01 14:44:59,147 INFO Class IActant available.
2024-06-01 14:44:59,149 INFO Class IAction available.
2024-06-01 14:44:59,149 INFO Class IAudit available.
2024-06-01 14:44:59,150 INFO Class IConcept available.
2024-06-01 14:44:59,151 INFO Class IEntity available.
2024-06-01 14:44:59,153 INFO Class IEvent available.
2024-06-01 14:44:59,154 INFO Class IGroup available.
2024-06-01 14:44:59,154 INFO Class ILabel available.
2024-06-01 14:44:59,155 INFO Class ILocation available.
2024-06-01 14:44:59,156 INFO Class IObject available.
2024-06-01 14:44:59,157 INFO Class IPerson available.
2024-06-01 14:44:59,158 INFO Class IProp available.
2024-06-01 14:44:59,159 INFO Class IReference available.
2024-06-01 14:44:59,159 INFO Class IResource available.
2024-06-01 14:44:59,161 INFO Class IStatement available.
2024-06-01 14:44:59,162 INFO Class ITerritory available.
2024-06-01 14:44:59,162 INFO Class IUser available.
2024-06-01 14:44:59,163 INFO Class IValue available.
2024-06-01 14:44:59,164

In [10]:
# initialisation of input data tables
tables = {}
header_infos = {}
entity_ids = {}

In [11]:
# load existing entities.json to create legacyId_to_uuid map
ddb1_entities = pd.read_json("./data/latest/entities.json")

In [12]:
legacyId_to_uuid = ddb1_entities[['legacyId','id']].copy().set_index('legacyId')
legacyId_to_uuid = legacyId_to_uuid[legacyId_to_uuid.index.notna()].to_dict()['id']

In [63]:
legacyId_to_uuid['T77-01-004']

'adb0ee87-9a91-40bd-ab1d-ffaa071d418a'

# Load input datasets

In [51]:
# partial load
# partial_load = ['R0075_persons', 'R0035_locations','R0083_events']
# partial_load = ['R0035_locations']
# partial_load = ['R0039_persons', 'R0040_locations','R0079_events']
# partial_load = ['R0076_persons', 'R0074_locations','R0104_events']

# R0087	Bologna Groups
# R0031	Bologna Locations
# R0030	Bologna Persons
# R0082	Bologna Trial Events
# partial_load = ['R0030_persons', 'R0031_locations','R0082_events']
# partial_load = ['R0104_events'] # 'R0030_persons', 'R0087_groups', ,'R0082_events'
# partial_load = ['R0030_persons','R0082_events'] # 'R0030_persons', 'R0087_groups', ,'R0082_events'
# partial_load = ['R0082_events']
partial_load = []

# empty value unifier
def unify_empty_value(df: pd.DataFrame, empty_values=None, unified_empty_value =''):
    if empty_values is None:
        empty_values = ["#N/A", "#VALUE!",'NA', 'NS'] # 'NA', 'NS',
    for naner in empty_values:
        df = df.replace(naner,unified_empty_value, regex=True)
    df. fillna(unified_empty_value, inplace=True)
    return df

# define cross-suffixes
casearea_to_entity_suffix = {
    "R0006" : {
        "P" : "R0006",
        "L" : "R0007",
        "E" : "R0008"
    },
    "R0007" : {
        "P" : "R0006",
        "L" : "R0007",
        "E" : "R0008"
    },
    "R0008" : {
        "P" : "R0006",
        "L" : "R0007",
        "E" : "R0008"
    },
    "R0075" : {
        "P" : "R0075",
        "L" : "R0035",
        "E" : "R0083",
        "G" : "R0105",
    },
    "R0035" : {
        "P" : "R0075",
        "L" : "R0035",
        "E" : "R0083",
        "G" : "R0105",
    },
    "R0083" : {
        "P" : "R0075",
        "L" : "R0035",
        "E" : "R0083",
        "G" : "R0105",
    },
    "R0105" : {
        "P" : "R0075",
        "L" : "R0035",
        "E" : "R0083",
        "G" : "R0105",
    },
    "R0087" : {
          "P" : "R0030",
          "L" : "R0031",
          "E" : "R0082",
          "G" : "R0087",
      },
    "R0030" : {
        "P" : "R0030",
        "L" : "R0031",
        "E" : "R0082",
        "G" : "R0087",
    },
    "R0031" : {
        "P" : "R0030",
        "L" : "R0031",
        "E" : "R0082",
        "G" : "R0087",
    },
    "R0082" : {
        "P" : "R0030",
        "L" : "R0031",
        "E" : "R0082",
        "G" : "R0087",
    },
    "R0039" : {
          "P" : "R0039",
          "L" : "R0040",
          "E" : "R0079",
      },
    "R0040" : {
          "P" : "R0039",
          "L" : "R0040",
          "E" : "R0079",
      },
    "R0079" : {
          "P" : "R0039",
          "L" : "R0040",
          "E" : "R0079",

      },
    "R0076" : {
          "P" : "R0076",
          "L" : "R0074",
          "E" : "R0104"
      },
    "R0074" : {
          "P" : "R0076",
          "L" : "R0074",
          "E" : "R0104"
      },
     "R0104" : {
          "P" : "R0076",
          "L" : "R0074",
          "E" : "R0104"
      },
}

def get_suffix_based_on_case_and_entity_type(string, case_suffix):
    # if valid entity type
    allowed_strict_entities = ['P','L','E','B','G']
    if any(string.startswith(c)for c in allowed_strict_entities):
        fl = string[0]
        return "_"+casearea_to_entity_suffix[case_suffix.replace("_","")][fl]
    else:
        return ""

def apply_suffix(string, case_suffix):
    if string in ['NS','NA']:
        return string
    mod_string = ""
    if "#" in string:
        strings = string.split("#")
        for s in strings:
            mod_string += " #"+ s.strip() + get_suffix_based_on_case_and_entity_type(s.strip(), case_suffix)
        mod_string = mod_string[2:]
    else:
        mod_string = string + get_suffix_based_on_case_and_entity_type(string, case_suffix)
    return mod_string


# load all input tables
load_sheets = {}
if len(partial_load) > 0:
    logger.info(f"Making PARTIAL LOAD with {partial_load}")
    for item in partial_load:
        load_sheets[item] = input_sheets[item]
else:
    load_sheets = input_sheets

for key, sheet in load_sheets.items():
    logger.info(f"Calling for {key} with sheet_name {sheet[0]}.")
    tables[key], header_infos[key] = d.load_df_from_gsheet(key,root_sheet_url + sheet[1], sheet[0], fromCache=False, header_in_row=sheet[2], clean=True, fillna=True, cleanByColumn="label", parse_hyperlink_formulas=True, numerize=False)
    tables[key] = unify_empty_value(tables[key])
    header_infos[key] = unify_empty_value(header_infos[key])

    # clean brackets
    for c in tables[key].columns:
      if "label" not in c:
        tables[key][c] = tables[key][c].replace("\[","",  regex=True)
        tables[key][c] = tables[key][c].replace("\]","",  regex=True)


    # case study dataset legacyId needs a suffix
    if "_" in key:
        columns = sheet[3]
        suffix = "_" + key.split("_")[0]
        for c in columns:
            # tables[key][c] = tables[key][c] + suffix
            logger.info(f"Using {key} and {c}.")
            tables[key][c]  = tables[key][c].apply(lambda item: apply_suffix(item,suffix) if len(str(item)) > 0 else item)

            #tables[key].loc[~tables[key][c].isna(), "c"] = tables[key][c] + "_" + suffix
            # df.loc[df["gender"] == "male", "gender"] = 1
            logger.info(f"{key} For columns {c} the suffix from {suffix} set was glued.")

    # code for legacyId copy and uuid creation
    # tables[key]['legacyId'] = tables[key]['id'].copy()
    tables[key].insert(2, 'legacyId', tables[key]['id'].copy())

    # inform instructive header about the new column and what to do with it
    header_infos[key]['legacyId'] = ""
    header_infos[key].at[3,'legacyId'] = "inside"
    tables[key]['id'] = tables[key].apply(lambda x: inkimplib2.get_uuid_id2(x,legacyId_to_uuid), axis=1)  # generate unique id for each row
    # make id dictionaries (it is much faster to search for keys there in legacyId>id retrievals)
    ed = tables[key][["legacyId","id"]].set_index("legacyId")

    if "_" in key:   # legacyIds are case-specified, i.e. all locations can be together in one table
        key = key.split("_")[1]

    if key in  entity_ids:
        entity_ids[key] = {**entity_ids[key],**ed["id"].to_dict()}
    else:
        entity_ids[key] = ed["id"].to_dict()

2024-06-01 19:49:50,601 INFO Calling for texts with sheet_name Texts.


20 2024-06-01 19:49:52 : Loading dataset texts
20 2024-06-01 19:49:52 : Opting for variant header at row 5.
20 2024-06-01 19:50:02 : Hyperlinks were detected and transformed in columns ['edition_1', 'edition_2', 'edition_3', 'persons_index_link', 'places_index_link'].
20 2024-06-01 19:50:02 : Dropping empty columns in the dataset texts : (1010, 120)
20 2024-06-01 19:50:02 : Deleted 821 empty rows by label.
20 2024-06-01 19:50:02 : Loaded and prepared dataset texts : (189, 120)


2024-06-01 19:50:02,459 WARNING legacy id T173 not found in ddb1 data
2024-06-01 19:50:02,465 WARNING legacy id T155 not found in ddb1 data
2024-06-01 19:50:02,485 WARNING legacy id T156 not found in ddb1 data
2024-06-01 19:50:02,486 WARNING legacy id T157 not found in ddb1 data
2024-06-01 19:50:02,486 WARNING legacy id T158 not found in ddb1 data
2024-06-01 19:50:02,487 WARNING legacy id T159 not found in ddb1 data
2024-06-01 19:50:02,487 WARNING legacy id T160 not found in ddb1 data
2024-06-01 19:50:02,487 WARNING legacy id T161 not found in ddb1 data
2024-06-01 19:50:02,488 WARNING legacy id T162 not found in ddb1 data
2024-06-01 19:50:02,489 WARNING legacy id T163 not found in ddb1 data
2024-06-01 19:50:02,490 WARNING legacy id T164 not found in ddb1 data
2024-06-01 19:50:02,490 WARNING legacy id T165 not found in ddb1 data
2024-06-01 19:50:02,491 WARNING legacy id T166 not found in ddb1 data
2024-06-01 19:50:02,491 WARNING legacy id T167 not found in ddb1 data
2024-06-01 19:50:02,

20 2024-06-01 19:50:03 : Loading dataset manuscripts
20 2024-06-01 19:50:03 : Opting for variant header at row 5.


2024-06-01 19:50:08,011 WARNING legacy id M163 not found in ddb1 data


...

2024-06-01 19:52:45,960 INFO Using R0074_locations and id.
2024-06-01 19:52:45,961 INFO R0074_locations For columns id the suffix from _R0074 set was glued.
2024-06-01 19:52:45,962 INFO Using R0074_locations and locationrelation1_value_id.
2024-06-01 19:52:45,963 INFO R0074_locations For columns locationrelation1_value_id the suffix from _R0074 set was glued.
2024-06-01 19:52:45,963 INFO Using R0074_locations and locationrelation2_value_id.
2024-06-01 19:52:45,964 INFO R0074_locations For columns locationrelation2_value_id the suffix from _R0074 set was glued.
2024-06-01 19:52:45,965 INFO Using R0074_locations and superordinate_location_id.
2024-06-01 19:52:45,965 INFO R0074_locations For columns superordinate_location_id the suffix from _R0074 set was glued.
2024-06-01 19:52:45,966 INFO Using R0074_locations and identificate_id.
2024-06-01 19:52:45,966 INFO R0074_locations For columns identificate_id the suffix from _R0074 set was glued.
2024-06-01 19:52:45,970 INFO Calling for R0104_

20 2024-06-01 19:52:45 : Hyperlinks were detected and transformed in columns [].
20 2024-06-01 19:52:45 : Dropping empty columns in the dataset R0074_locations : (1000, 36)
20 2024-06-01 19:52:45 : Deleted 947 empty rows by label.
20 2024-06-01 19:52:45 : Loaded and prepared dataset R0074_locations : (53, 36)
20 2024-06-01 19:52:47 : Loading dataset R0104_events
20 2024-06-01 19:52:47 : Opting for variant header at row 5.


2024-06-01 19:52:54,511 INFO Using R0104_events and id.
2024-06-01 19:52:54,512 INFO R0104_events For columns id the suffix from _R0104 set was glued.
2024-06-01 19:52:54,513 INFO Using R0104_events and participant_id.
2024-06-01 19:52:54,514 INFO R0104_events For columns participant_id the suffix from _R0104 set was glued.
2024-06-01 19:52:54,515 INFO Using R0104_events and inquisitor_id.
2024-06-01 19:52:54,515 INFO R0104_events For columns inquisitor_id the suffix from _R0104 set was glued.
2024-06-01 19:52:54,516 INFO Using R0104_events and notary_id.
2024-06-01 19:52:54,516 INFO R0104_events For columns notary_id the suffix from _R0104 set was glued.


20 2024-06-01 19:52:54 : Hyperlinks were detected and transformed in columns [].
20 2024-06-01 19:52:54 : Dropping empty columns in the dataset R0104_events : (1623, 57)
20 2024-06-01 19:52:54 : Deleted 1562 empty rows by label.
20 2024-06-01 19:52:54 : Loaded and prepared dataset R0104_events : (61, 57)


2024-06-01 19:52:54,517 INFO Using R0104_events and witness_id.
2024-06-01 19:52:54,518 INFO R0104_events For columns witness_id the suffix from _R0104 set was glued.
2024-06-01 19:52:54,518 INFO Using R0104_events and spatial_ref_value_id.
2024-06-01 19:52:54,518 INFO R0104_events For columns spatial_ref_value_id the suffix from _R0104 set was glued.
2024-06-01 19:52:54,519 INFO Using R0104_events and temporal_ref_1_value_id.
2024-06-01 19:52:54,519 INFO R0104_events For columns temporal_ref_1_value_id the suffix from _R0104 set was glued.
2024-06-01 19:52:54,520 INFO Using R0104_events and temporal_ref_2_value_id.
2024-06-01 19:52:54,521 INFO R0104_events For columns temporal_ref_2_value_id the suffix from _R0104 set was glued.
2024-06-01 19:52:54,521 INFO Using R0104_events and accuser_id.
2024-06-01 19:52:54,522 INFO R0104_events For columns accuser_id the suffix from _R0104 set was glued.
2024-06-01 19:52:54,522 INFO Using R0104_events and accused_id.
2024-06-01 19:52:54,523 INFO 

In [286]:
# making empty table of values
tables['values'] = pd.DataFrame(columns=['id','value','origin'])
tables['locations'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['events'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['props'] = pd.DataFrame(columns=['id','type','type_id','value','value_id','original_field','origin','entityId','legacyId'])
tables['relations'] = pd.DataFrame(columns=['id','type','logic','certainty','entityIds','origin'])

In [21]:
tables.keys(), entity_ids.keys()

(dict_keys(['texts', 'manuscripts', 'resources', 'actions', 'concepts', 'R0006_persons', 'R0007_locations', 'R0008_events', 'R0075_persons', 'R0035_locations', 'R0083_events', 'R0105_groups', 'R0030_persons', 'R0087_groups', 'R0031_locations', 'R0082_events', 'R0039_persons', 'R0040_locations', 'R0079_events', 'R0076_persons', 'R0074_locations', 'R0104_events']),
 dict_keys(['texts', 'manuscripts', 'resources', 'actions', 'concepts', 'persons', 'locations', 'events', 'groups']))

In [52]:
#inkimplib2.missing_legacyIds_in_ddb1
#inkimplib2.explicit_uuids_in_import
inkimplib2.new_uuids_in_import

len(inkimplib2.missing_legacyIds_in_ddb1), len(inkimplib2.explicit_uuids_in_import), len(inkimplib2.new_uuids_in_import)

(212, 50, 0)

In [57]:
pd.Series(inkimplib2.missing_legacyIds_in_ddb1,name="missing_in_ddb1").to_csv("missing_in_ddb1.csv")
pd.Series(inkimplib2.explicit_uuids_in_import,name="explcit_uuid_ddb1").to_csv("explicit_uuid_ddb1.csv")

In [58]:
"P2116_R0030" in inkimplib2.missing_legacyIds_in_ddb1

True

In [23]:
header_infos['R0082_events']

,id,label,label_language,entity_logical_type,resource_id,document_no,page_range_starts,page_range_ends,page_range_concatenated,class_id,...,accused_label,mentioned_id,mentioned_label,denied_id,denied_label,note,editor,done,checked,legacyId
0,,,,,,,,,,,...,,,,,,"Save as the multi-note (if ""#"" is present, sep...",audits,,,
1,,,,,,,,,id,id,...,,id,,id,,,,,,
2,,,,,,,,,R0034,Classification,...,,C2924,,C3403,,note,,,,
3,inside,inside,inside,inside,discard,discard,discard,discard,reference_part,relation,...,discard,propvalue,discard,propvalue,discard,inside,special,discard,discard,inside


In [16]:
# I AM AN IDIOT, I have overwritten this old data... :(

# Save a dictionary into a pickle file.
import dill
# pickle.dump( tables, open( "./data/import-input-tables-canonical.pickle", "wb" ) )
#pickle.dump( header_infos, open( "./data/import-input-headers-canonical.pickle", "wb" ) )
#pickle.dump( dh.entity_ids, open( "./data/import-input-entity_ids-canonical.pickle", "wb" ) )
#pickle.dump( entity_ids, open( "./data/import-input-entity_ids-canonical.pickle", "wb" ) )

In [17]:
pickle.dump( tables, open( "./data/2024/import-input-tables.pickle", "wb" ) )
pickle.dump( header_infos, open( "./data/2024/import-input-headers.pickle", "wb" ) )
#pickle.dump( dh.entity_ids, open( "./data/import-input-entity_ids-canonical.pickle", "wb" ) )
pickle.dump( entity_ids, open( "./data/2024/import-input-entity_ids.pickle", "wb" ) )

In [ ]:
# load from pickle file
# tables = pickle.load( open( "./data/import-input-tables.pickle", "rb" ) )
# header_infos = pickle.load( open( "./data/import-input-headers.pickle", "rb" ) )
# entity_ids = pickle.load( open( "./data/import-input-entity_ids.pickle", "rb" ) )

In [18]:
entity_ids.keys()

dict_keys(['texts', 'manuscripts', 'resources', 'actions', 'concepts', 'persons', 'locations', 'events', 'groups'])

In [20]:
# manual corrections in the input data

### Responsible editors

#### Data
Only concepts have "editors" filled.

#### Specification

Only entities have audits, i.e. the prop object, the ref object don't. (Could be wrong?)

**Rationale:**  Each entity will have 2 audit creation records. One with the user "import", second with the user "responsible editor.

Implementaions possiblities:
  - make the audits during the parsing : the way
  - make the audits afer the parsing is done (for secondary created entities, I will not have the context, and will be problem to find a responsible editor
  -

In [19]:
# the names in the input tables  vs their user ids
# the users are imported through the user.json file

#David Zbíral	david.zbiral@mail.muni.cz
#Robert Shaw	robert.shaw@mail.muni.cz
#Jan Král	kral.jan@mail.muni.cz
#Reima Välimäki	reima.valimaki@gmail.com
#Lidia Hinz-Wieczorek	lidhin93@gmail.com
#Davor Salihović	davor.salihovic@mail.muni.cz
#Katia Riccardo	katia.riccardo@mail.muni.cz
#Larissa de Freitas Lyth	larissa.lyth@mail.muni.cz
#Larissa de Freitas Lyth

editors = {"import":"0", "David Zbíral":"100", "Robert Shaw":"101", "Davor Salihović":"102", "Katia Riccardo":"103", "Jan Král":"104", "Reima Välimäki":"105","Lidia Hinz-Wieczorek":"106", "Larissa de Freitas Lyth":"107"}

# Run encapsulated

In [9]:
# load clean input from pickle file
tables = pickle.load( open( "./data/import-input-tables.pickle", "rb" ) )
header_infos = pickle.load( open( "./data/import-input-headers.pickle", "rb" ) )
entity_ids = pickle.load( open( "./data/import-input-entity_ids.pickle", "rb" ) )

In [20]:
# hacking the content
tables['texts'].loc[tables['texts']['legacyId']=='T175','label_language'] = "English"
tables['resources']['label_language'] = "English"
tables['concepts'].loc[tables['concepts']['status']=='bs','status'] = "approved"
tables['R0031_locations'].loc[tables['R0031_locations']['id']=='L0647','coordinates_concatenated'] = "45.46617; 10.60522"


# 2024-06-01 13:59:17,312 ERROR Cannot get entity id '' from value asdf  in special>:creation_event_id E0145_M163LegacyId: M163.

In [21]:
# exploring the content
tables.keys()

dict_keys(['texts', 'manuscripts', 'resources', 'actions', 'concepts', 'R0006_persons', 'R0007_locations', 'R0008_events', 'R0075_persons', 'R0035_locations', 'R0083_events', 'R0105_groups', 'R0030_persons', 'R0087_groups', 'R0031_locations', 'R0082_events', 'R0039_persons', 'R0040_locations', 'R0079_events', 'R0076_persons', 'R0074_locations', 'R0104_events'])

In [59]:
# dh contain input data, but also generated data (aka generated resource) and additional output (relatins, audits ...)
dh = inkimplib2.DataHolder() # singleton
dh.pass_data(tables, header_infos, entity_ids, table_to_entity, editors)
dh.reset_generated_collection()

2024-06-01 20:17:29,781 INFO Class IActant available.
2024-06-01 20:17:29,782 INFO Class IAction available.
2024-06-01 20:17:29,783 INFO Class IAudit available.
2024-06-01 20:17:29,785 INFO Class IConcept available.
2024-06-01 20:17:29,786 INFO Class IEntity available.
2024-06-01 20:17:29,787 INFO Class IEvent available.
2024-06-01 20:17:29,788 INFO Class IGroup available.
2024-06-01 20:17:29,788 INFO Class ILabel available.
2024-06-01 20:17:29,790 INFO Class ILocation available.
2024-06-01 20:17:29,792 INFO Class IObject available.
2024-06-01 20:17:29,793 INFO Class IPerson available.
2024-06-01 20:17:29,794 INFO Class IProp available.
2024-06-01 20:17:29,795 INFO Class IReference available.
2024-06-01 20:17:29,796 INFO Class IResource available.
2024-06-01 20:17:29,797 INFO Class IStatement available.
2024-06-01 20:17:29,798 INFO Class ITerritory available.
2024-06-01 20:17:29,799 INFO Class IUser available.
2024-06-01 20:17:29,801 INFO Class IValue available.
2024-06-01 20:17:29,801

In [26]:
dh.reset_generated_collection()

In [48]:
logger.info(f"Start")
cp = inkimplib2.ParseController(entity_list=['texts','actions', 'manuscripts','resources','concepts'])
cp.parse()
logger.info(f"End")

2024-05-27 18:43:12,312 INFO Start
2024-05-27 18:43:12,315 INFO TEXTS Uncertain parsing instructions in 0 columns: .
2024-05-27 18:43:12,337 INFO TEXTS 66 columns have been dropped (discard:66, unknown:0: []). Table now has 54 columns, inside:9,propvalue:15, special:6, proptype: 4, proptype_2nd: 0, dependent:0, reference_part:0. Originally 120 columns.
2024-05-27 18:43:12,339 INFO ACTIONS Uncertain parsing instructions in 0 columns: .
2024-05-27 18:43:12,365 INFO ACTIONS 64 columns have been dropped (discard:63, unknown:1: ['ddb1_uuid']). Table now has 25 columns, inside:14,propvalue:0, special:1, proptype: 0, proptype_2nd: 0, dependent:0, reference_part:2. Originally 89 columns.
2024-05-27 18:43:12,367 INFO MANUSCRIPTS Uncertain parsing instructions in 0 columns: .
2024-05-27 18:43:12,377 INFO MANUSCRIPTS 35 columns have been dropped (discard:35, unknown:0: []). Table now has 32 columns, inside:8,propvalue:8, special:5, proptype: 4, proptype_2nd: 0, dependent:0, reference_part:0. Orig

In [246]:
dh.tables['resources'][dh.tables['resources']['label'].str.contains("Douais")]

,id,label,legacyId,detail,label_language,for_migration_to_ddb1,do_before_migration,do_after_migration,migration_check_by_david_done,ready_for_migration,...,parsing_type,parsing_entity,dissinet_repository_url,spreadsheet_id,sheet_name,note,editor,parsing_rows_explained,dissinet_respository_url,origin
134,1afca8bb-aadb-478e-a346-b96be9ae05b2,"Douais, 1900",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zotero.org/groups/446972/david_zbi...,"special C1188>:edition_1Douais, 1900|https://w..."
151,0f32b8b0-9295-494f-9d06-d28004b78804,"Douais, 1900",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zotero.org/groups/446972/david_zbi...,"special C1188>:edition_2Douais, 1900|https://w..."
164,08e99be1-5227-49eb-aa4d-4e936e3cbd00,"Douais, 1900",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zotero.org/groups/446972/david_zbi...,"special C1188>:edition_1Douais, 1900|https://w..."
165,90b15e22-bfd6-4c3f-b250-f819e5a41a8d,"Douais, 1900",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zotero.org/groups/446972/david_zbi...,"special C1188>:edition_1Douais, 1900|https://w..."


In [165]:
# save specific state of the dh object
import dill as pickle
pickle.dump(dh, open("./data/dh.with.meta.canonical-17-1.pickle","wb"))
# saved also ./data/dh.with.meta.canonical.pickle, 13.1. 2022 - theoretically definitive meta entities (texts, actions, manuscripts, resources, concepts) import
# saved also ./data/dh.with.meta.canonical-17-1.pickle, 17.1. 2022 - theoretically definitive meta entities (texts, actions, manuscripts, resources, concepts) import, with correct input case area datasets

In [332]:
# load specific state of the dh object
import dill as pickle
dh = pickle.load(open( "./data/dh.with.meta.canonical-17-1.pickle", "rb" ))    # contains errors in bologna ids :(

In [333]:
for key, table in tables.items():
  rewrite = ['texts','resources','actions','conpcets', 'manuscripts']

  if "_" in key or key in rewrite:
    dh.tables[key] = tables[key]
    dh.header_infos[key] = header_infos[key]

In [334]:
dh.checkIDs(fix_difference=True)

############## texts
Data group: texts
############## texts 0.
############## manuscripts
Data group: manuscripts
############## manuscripts 0.
############## resources
Data group: resources
############## resources 0.
############## actions
Data group: actions
############## actions 0.
############## concepts
Data group: concepts
############## concepts 0.
############## R0006_persons
Data group: persons
############## R0006_persons 0.
############## R0007_locations
Data group: locations
############## R0007_locations 0.
############## R0008_events
Data group: events
############## R0008_events 0.
############## R0075_persons
Data group: persons
############## R0075_persons 0.
############## R0035_locations
Data group: locations
############## R0035_locations 0.
############## R0083_events
Data group: events
############## R0083_events 0.
############## R0105_groups
Data group: groups
############## R0105_groups 0.
############## R0030_persons
Data group: persons
############## R0030_

In [27]:
dh.reset_generated_collection()

2024-06-01 14:10:52,933 INFO Class IActant available.
2024-06-01 14:10:52,934 INFO Class IAction available.
2024-06-01 14:10:52,935 INFO Class IAudit available.
2024-06-01 14:10:52,936 INFO Class IConcept available.
2024-06-01 14:10:52,937 INFO Class IEntity available.
2024-06-01 14:10:52,938 INFO Class IEvent available.
2024-06-01 14:10:52,939 INFO Class IGroup available.
2024-06-01 14:10:52,940 INFO Class ILabel available.
2024-06-01 14:10:52,941 INFO Class ILocation available.
2024-06-01 14:10:52,942 INFO Class IObject available.
2024-06-01 14:10:52,943 INFO Class IPerson available.
2024-06-01 14:10:52,943 INFO Class IProp available.
2024-06-01 14:10:52,944 INFO Class IReference available.
2024-06-01 14:10:52,945 INFO Class IResource available.
2024-06-01 14:10:52,946 INFO Class IStatement available.
2024-06-01 14:10:52,947 INFO Class ITerritory available.
2024-06-01 14:10:52,948 INFO Class IUser available.
2024-06-01 14:10:52,949 INFO Class IValue available.
2024-06-01 14:10:52,950

In [60]:
logger.info(f"Start")
# Guglielmites
# cp = inkimplib.ParseController(entity_list=[ 'R0006_persons', 'R0007_locations', 'R0008_events']) # ,'R0008_events'
# 'R0006_persons',

# Castellario
# cp = inkimplib.ParseController(entity_list=['R0075_persons', 'R0105_groups', 'R0035_locations','R0083_events'])

# Bologna
# cp = inkimplib2.ParseController(entity_list=['R0030_persons','R0082_events','R0031_locations']) # 'R0030_persons','R0031_locations',

# Carcassone # R0040_locations, R0039_persons
# cp = inkimplib.ParseController(entity_list=['R0039_persons', 'R0040_locations', 'R0079_events'])

#Casasco
# cp = inkimplib.ParseController(entity_list=['R0076_persons', 'R0074_locations','R0104_events'])

# all
# withou Bologna, 'R0030_persons', 'R0087_groups', 'R0031_locations','R0082_events',
# cp = inkimplib.ParseController(entity_list=['R0006_persons', 'R0007_locations','R0008_events', 'R0075_persons', 'R0105_groups', 'R0035_locations','R0083_events',  'R0039_persons', 'R0040_locations', 'R0079_events','R0076_persons', 'R0074_locations','R0104_events'])

#cp = inkimplib.ParseController(entity_list=['texts','actions', 'manuscripts','resources','concepts','R0006_persons', 'R0007_locations','R0008_events', 'R0075_persons', 'R0105_groups', 'R0035_locations','R0083_events',  'R0039_persons', 'R0040_locations', 'R0079_events','R0076_persons', 'R0074_locations','R0104_events','R0030_persons', 'R0087_groups', 'R0031_locations','R0082_events'], dh=dh)

# full REIMPORT parsing batch
#cp = inkimplib2.ParseController(entity_list=['texts','actions', 'manuscripts','resources','concepts','R0031_locations','R0030_persons','R0082_events'], dh=dh) 
cp = inkimplib2.ParseController(entity_list=['R0031_locations','R0030_persons','R0082_events'], dh=dh) 

cp.parse()
logger.info(f"End")

2024-06-01 20:18:02,344 INFO Start
2024-06-01 20:18:02,347 INFO R0031_LOCATIONS Uncertain parsing instructions in 0 columns: .
2024-06-01 20:18:02,372 INFO R0031_LOCATIONS 31 columns have been dropped (discard:31, unknown:0: []). Table now has 26 columns, inside:8,propvalue:2, special:4, proptype: 1, proptype_2nd: 1, dependent:0, reference_part:1. Originally 57 columns.
2024-06-01 20:18:02,374 INFO R0030_PERSONS Uncertain parsing instructions in 0 columns: .
2024-06-01 20:18:02,478 INFO R0030_PERSONS 52 columns have been dropped (discard:52, unknown:0: []). Table now has 41 columns, inside:6,propvalue:26, special:1, proptype: 5, proptype_2nd: 1, dependent:0, reference_part:1. Originally 93 columns.
2024-06-01 20:18:02,481 INFO R0082_EVENTS Uncertain parsing instructions in 0 columns: .
2024-06-01 20:18:02,518 INFO R0082_EVENTS 36 columns have been dropped (discard:35, unknown:1: ['liber_securitatum (auto accusations tumult  13.05.1299)']). Table now has 34 columns, inside:6,propvalue:1

In [61]:
# save to resulted dh
pickle.dump(dh, open("./data/2024/dh.import-reimport-bologna.pickle","wb"))

In [64]:
pd.DataFrame(dh.additional_entities)

,class,id,label,language,detail,data,props,notes,status,references,legacyId
0,L,a9afebca-b6fd-43eb-beaf-838a469e03e9,Agnanum,lat,,{'logicalType': 'definite'},[],[Import batch [ReImport 06/2024] 2024-06-01 20...,1,[],
1,L,d41a087f-5ffc-436d-aeaf-2435f3a50145,Aglanum,lat,,{'logicalType': 'definite'},[],[Import batch [ReImport 06/2024] 2024-06-01 20...,1,[],
2,L,0c02d7da-6971-41f9-96bc-d2a8ca0c1d8d,Agnana,ita,,{'logicalType': '1'},[],[Import batch [ReImport 06/2024] 2024-06-01 20...,1,[],
3,V,d5f1ee4e-6a5e-45da-aba4-b279d0f2ae8f,488,,,{'logicalType': '1'},[],[Import batch [ReImport 06/2024] 2024-06-01 20...,1,[],NaN
4,L,ff591ed3-b31f-47dc-989d-b3b189e80127,Agnana,ita,,{'logicalType': '1'},[],[Import batch [ReImport 06/2024] 2024-06-01 20...,1,[],
...,...,...,...,...,...,...,...,...,...,...,...
11692,T,9dc9445e-7e7e-43c0-a96a-cd27c03fb32c,"Sentenza assolutoria di Benvenutus, rector S.P...",ita,,{'parent': {'territoryId': '75a452d6-81c9-476f...,"[{'bundleEnd': False, 'bundleStart': False, 'c...",[Import batch [ReImport 06/2024] 2024-06-01 20...,1,[],T77-19-921
11693,V,8715e886-8529-4c27-98a9-b4d550d05b98,1309-10-16,,,{'logicalType': '1'},[],[Import batch [ReImport 06/2024] 2024-06-01 20...,1,[],NaN
11694,V,f410cd30-0ee0-4506-ac7d-280427cab603,713-714,,,{'logicalType': '1'},[],[Import batch [ReImport 06/2024] 2024-06-01 20...,1,[],NaN
11695,T,d589fe1c-bdf6-44a1-9124-3c5f88eae73e,Sentenza assolutoria di Petrus Mutinensis,ita,,{'parent': {'territoryId': '75a452d6-81c9-476f...,"[{'bundleEnd': False, 'bundleStart': False, 'c...",[Import batch [ReImport 06/2024] 2024-06-01 20...,1,[],T77-19-922


In [ ]:
# cp = ParseController(entity_list=['texts','actions', 'manuscripts','resources','concepts','R0006_persons', 'R0007_locations','R0008_events'])
# cp = ParseController(entity_list=['texts','actions', 'manuscripts','resources','concepts','R0006_persons', 'R0007_locations','R0008_events',"R0075_persons","R0035_locations","R0083_events"])

# cp = ParseController(entity_list=['texts','actions', 'manuscripts','resources','concepts','R0006_persons', 'R0007_locations','R0008_events',"R0075_persons","R0035_locations","R0083_events"], existing_parser=cp, reparse_entity_list=['R0035_locations'])

# cp = ParseController(entity_list=['texts', "concepts", "R0008_events"])
#cp = ParseController(entity_list=[ "R0007_locations"])
# , 'R0006_persons', 'R0007_locations'
# cp = ParseController(entity_list=['texts','manuscripts','resources','concepts','actions','R0006_persons', 'R0007_locations']) # , 'R0006_person \as', 'R0007_locations'
# cp = ParseController(entity_list=['R0006_persons', 'R0007_locations'])
#cp = ParseController(entity_list=['R0008_events'])
#cp = ParseController(entity_list=['actions','concepts'])

# After parsing

### Merging reciprocal relation records
Antonym, PropertyReciprocal, SubjectActant1Reciprocal, Synonym, Related

In [65]:
# ANT, PRR, SAR, SYN, REL
def getE(eIds):
    eids = eIds.copy() # has to be copy, otherwise it modifies original relation_records list-arrays
    eids.sort()
    return eids[0]+"!"+eids[1]

rdf = pd.DataFrame(dh.relation_records)
rdf['eids'] = rdf['entityIds'].apply(lambda row:getE(row))
removeRR  = rdf[rdf[['type','eids']].duplicated()].sort_values('eids')['id'].to_list()
removeRR

['059a0a8c-78e6-4e36-8c90-b3c5d8db3b9d',
 '2c208efe-52ac-4aa5-889e-5f70402a114b',
 '94779b91-e156-4669-994a-d213bfd8c6a2',
 '2f29f187-beb9-483b-9e15-e76b80e2ff36']

In [66]:
relation_records_cleaned = []
for item in dh.relation_records:
    if item['id'] not in removeRR:
        relation_records_cleaned.append(item)

In [67]:
len(dh.relation_records), len(relation_records_cleaned)

(5712, 5708)

In [40]:
# check for relations entityIds
# a=0
# for item in relation_records_cleaned:
#     #if "81e38bb2-6f84-4b30-acac-b6931929dbaf" == item['id']:
#     if "7db804db-fa2d-4624-810d-b3440e2f4001" in item['entityIds'][0] and item['type']=="SCL":
#         a += 1
#         print(item)
# print(a)

{'id': 'dbb07e20-a1c6-49a0-a47e-dfde606f8e88', 'type': 'SCL', 'entityIds': ['7db804db-fa2d-4624-810d-b3440e2f4001', '479cc4f3-b877-44a5-a0dd-c6a7cc813163']}
1


In [73]:
# test

dh.tables['resources'][dh.tables['resources']['label'].str.contains("Paris")]

,id,label,legacyId,ddb1_uuid,detail,label_language,for_migration_to_ddb1,reason_for_migration_decision,do_before_migration,check_immediately_after_migration,...,component_label,parsing_type,parsing_entity,dissinet_repository_url,spreadsheet_id,sheet_name,note,editor,parsing_rows_explained,origin
132,07d993a6-3711-4343-b337-730baab2e492,Johannes monachus ad Olibam in: André de Fleur...,NaN,NaN,NaN,English,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,special C1188>:edition_1Johannes monachus ad O...
135,b6f36455-1cf5-45cd-ac82-f432c6a61be4,"Adémar de Chabannes, Chronique, ed. J. Chavano...",NaN,NaN,NaN,English,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,"special C1188>:edition_2Adémar de Chabannes, C..."
138,6f30d094-dd08-40b0-8596-d403ce16167e,"André de Fleury, ""Vie de Gauzlin, abbé de Fleu...",NaN,NaN,NaN,English,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,"special C1188>:edition_1André de Fleury, ""Vie ..."
141,6f6471eb-1b16-47f6-ac64-b8bf7cba6088,"Paul de Saint-Père de Chartres, ""Monasterii Sa...",NaN,NaN,NaN,English,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,special C1188>:edition_1Paul de Saint-Père de ...
192,2dbe0db5-a805-48b1-a6b7-7e3bb0b3e812,"Reproduction of Paris, Bibliothèque nationale ...",NaN,NaN,NaN,English,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://gallica.bnf.fr/ark:/12148/btv1b10035999g,NaN,NaN,NaN,NaN,NaN,Making Resource entity 'link|https://gallica.b...
193,62fa7ea3-2dc6-465c-af0a-fccd0e6a3320,"Reproduction of Paris, Bibliothèque nationale ...",NaN,NaN,NaN,English,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://gallica.bnf.fr/ark:/12148/btv1b10036000n,NaN,NaN,NaN,NaN,NaN,Making Resource entity 'link|https://gallica.b...
194,4f0926cb-0530-47b6-a46c-fc304e1c3bd0,"Reproduction of Paris, Bibliothèque nationale ...",NaN,NaN,NaN,English,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://gallica.bnf.fr/ark:/12148/btv1b100365190,NaN,NaN,NaN,NaN,NaN,Making Resource entity 'link|https://gallica.b...
195,82980f47-8f05-41bd-aa3f-c24620fdc96f,"Reproduction of Paris, Bibliothèque nationale ...",NaN,NaN,NaN,English,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://gallica.bnf.fr/ark:/12148/btv1b100361092,NaN,NaN,NaN,NaN,NaN,Making Resource entity 'link|https://gallica.b...
197,03386182-e196-4f0a-99e1-28d212cd9a26,"Reproduction of Paris, Bibliothèque nationale ...",NaN,NaN,NaN,English,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://gallica.bnf.fr/ark:/12148/btv1b100338354,NaN,NaN,NaN,NaN,NaN,Making Resource entity 'link|https://gallica.b...
199,ff23b963-2379-4354-884e-f50718033f56,"Reproduction of Paris, Bibliothèque nationale ...",NaN,NaN,NaN,English,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://gallica.bnf.fr/ark:/12148/btv1b5315700...,NaN,NaN,NaN,NaN,NaN,Making Resource entity 'link|https://gallica.b...


In [315]:
dh.entity_ids['resources']['M21']

'3feae15a-e17a-4844-b6a3-88678a4720db'

In [316]:
dh.checkIDs()

############## texts
Data group: texts
############## texts 0.
############## manuscripts
Data group: manuscripts
############## manuscripts 0.
############## resources
There are duplicates! 107    NaN
108    NaN
109    NaN
110    NaN
111    NaN
      ... 
180    NaN
181    NaN
182    NaN
183    NaN
184    NaN
Name: legacyId, Length: 78, dtype: object
Data group: resources
############## resources 0.
############## actions
Data group: actions
############## actions 0.
############## concepts
Data group: concepts
############## concepts 0.
############## R0006_persons
Data group: persons
############## R0006_persons 0.
############## R0007_locations
Data group: locations
############## R0007_locations 0.
############## R0008_events
Data group: events
############## R0008_events 0.
############## R0075_persons
Data group: persons
############## R0075_persons 0.
############## R0035_locations
Data group: locations
############## R0035_locations 0.
############## R0083_events
Data group: e

### Profiling

In [ ]:
#%reload_ext line_profiler

# def profile():
#    cp.parsers['concepts'].parse_rows()

#%lprun -f EntityMapper.get_entity_id profile()

In [ ]:
# %prun -s tottime profile()

In [ ]:
# %lprun?

### Last checks

In [44]:
logger.info(f"There are {len(dh.additional_entities)} additionally created entities (e.g. values, resources ...).")

2024-06-01 17:14:41,874 INFO There are 14544 additionally created entities (e.g. values, resources ...).


# Output the parsed data

In [68]:
# root data
root_territory = """
 {
    "id": "T0",
    "class": "T",
    "data": { "parent": false },
    "label": "root",
    "detail": "",
    "language": "lat",
    "notes": [],
    "props": []
  }
"""

In [70]:
# reimport output

r_entities = ""
r_relations = ""
r_audits = ""
# entities_content = ""

# save json object list of the parsed entities from google sheets in the text file
with open('../datasets/reimport/new_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(dh.js_objects, f)

# save json object list of the "newly created entities" in the text file
with open('../datasets/reimport/additional_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(dh.additional_entities, f)


entities_content  = ""

# read  entities.json
with open('../datasets/reimport/new_entities.json','r') as f:
    #entities_content = f.readlines()
    new_entities_content = f.read().replace('\n', '')

additional_entities_string = json.dumps(dh.additional_entities)

# write just the new parse data to the entities json.
with open('../datasets/reimport/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content =  "[" + root_territory + "," + additional_entities_string[1:-1]+ ","+  new_entities_content[1:]
    f.write(str(merge_content))

relations_entities_string = json.dumps(relation_records_cleaned)
audits_entities_string = json.dumps(dh.audits)

# write relations
with open('../datasets/reimport/relations.json','w', encoding='utf-8') as f:
    merge_content =  relations_entities_string[0:-1] + "," + r_relations + "]"
    f.write(str(merge_content))
    #json.dump(relation_records_cleaned, f)

# write the audits json.
with open('../datasets/reimport/audits.json','w', encoding='utf-8') as f:
    merge_content =  audits_entities_string[0:-1] + "]"
    f.write(str(merge_content))
    #json.dump(dh.audits, f)



logger.info("END reimport output")


2024-06-01 23:26:36,609 INFO END reimport output


In [355]:
# open robert's example of Niort coding
robert_data1317 = {
  "entities": json.load(open("../datasets/experiments/diff1317/entities.json","r")),
  "relations": json.load(open("../datasets/experiments/diff1317/relations.json","r")),
  "audits": json.load(open("../datasets/experiments/diff1317/audits.json","r"))
}

robert_data1719 = {
  "entities": json.load(open("../datasets/experiments/diff1719/entities.json","r")),
  "relations": json.load(open("../datasets/experiments/diff1719/relations.json","r")),
  "audits": json.load(open("../datasets/experiments/diff1719/audits.json","r"))
}

robert_data1920 = {
  "entities": json.load(open("../datasets/experiments/diff1920/entities.json","r")),
  "relations": json.load(open("../datasets/experiments/diff1920/relations.json","r")),
  "audits": json.load(open("../datasets/experiments/diff1920/audits.json","r"))
}

robert_data2021 = {
  "entities": json.load(open("../datasets/experiments/diff2021/entities.json","r")),
  "relations": json.load(open("../datasets/experiments/diff2021/relations.json","r")),
  "audits": json.load(open("../datasets/experiments/diff2021/audits.json","r"))
}


r_entities1317 = json.dumps(robert_data1317["entities"])[1:-1]
r_entities1719 = json.dumps(robert_data1719["entities"])[1:-1]
r_entities1920 = json.dumps(robert_data1920["entities"])[1:-1]
r_entities2021 = json.dumps(robert_data2021["entities"])[1:-1]

r_relations1317 = json.dumps(robert_data1317['relations'])[1:-1]
r_relations1719 = json.dumps(robert_data1719['relations'])[1:-1]
r_relations1920 = json.dumps(robert_data1920['relations'])[1:-1]
r_relations2021 = json.dumps(robert_data2021['relations'])[1:-1]
#r_audits = json.dumps(robert_data['audits'])[1:len(str(robert_data['audits']))-1]

In [356]:
# ideal "2023-01-13T20:05:56Z"
import datetime
for item in robert_data1317["audits"]:
  #print(item['date']['epoch_time']), print(datetime.datetime.fromtimestamp(item['date']['epoch_time']))
  new_time_chunks = str(datetime.datetime.fromtimestamp(item['date']['epoch_time'])).split(" ")
  new_time = new_time_chunks[0]+"T"+new_time_chunks[1].split(".")[0]+"Z"
  item['date'] = new_time

r_audits1317 = json.dumps(robert_data1317['audits'])[1:-1]

for item in robert_data1719["audits"]:
  #print(item['date']['epoch_time']), print(datetime.datetime.fromtimestamp(item['date']['epoch_time']))
  new_time_chunks = str(datetime.datetime.fromtimestamp(item['date']['epoch_time'])).split(" ")
  new_time = new_time_chunks[0]+"T"+new_time_chunks[1].split(".")[0]+"Z"
  item['date'] = new_time

r_audits1719 = json.dumps(robert_data1719['audits'])[1:-1]

for item in robert_data1920["audits"]:
  #print(item['date']['epoch_time']), print(datetime.datetime.fromtimestamp(item['date']['epoch_time']))
  new_time_chunks = str(datetime.datetime.fromtimestamp(item['date']['epoch_time'])).split(" ")
  new_time = new_time_chunks[0]+"T"+new_time_chunks[1].split(".")[0]+"Z"
  item['date'] = new_time

r_audits1920 = json.dumps(robert_data1920['audits'])[1:-1]

for item in robert_data2021["audits"]:
  #print(item['date']['epoch_time']), print(datetime.datetime.fromtimestamp(item['date']['epoch_time']))
  new_time_chunks = str(datetime.datetime.fromtimestamp(item['date']['epoch_time'])).split(" ")
  new_time = new_time_chunks[0]+"T"+new_time_chunks[1].split(".")[0]+"Z"
  item['date'] = new_time

r_audits2021 = json.dumps(robert_data2021['audits'])[1:-1]

In [357]:
len(r_entities1317), len(r_entities1719),len(r_entities1920),len(r_entities2021)

(35168, 54585, 31748, 7110)

In [490]:
r_entities = r_entities1317 + "," + r_entities1719 + "," + r_entities1920 + "," + r_entities2021
r_relations = r_relations1317 + "," + r_relations1719 + "," + r_relations1920 + "," + r_relations2021
r_audits = r_audits1317 + "," + r_audits1719 + "," + r_audits1920 + "," + r_audits2021

In [492]:

# save json object list of the parsed entities from google sheets in the text file
with open('../datasets/all-test/new_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(dh.js_objects, f)

# save json object list of the "newly created entities" in the text file
with open('../datasets/all-test/additional_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(dh.additional_entities, f)

# read  entities.json
with open('../datasets/all/entities.json','r') as f:
    #entities_content = f.readlines()
    entities_content = f.read().replace('\n', '')

# read  entities.json
with open('../datasets/all-test/new_entities.json','r') as f:
    #entities_content = f.readlines()
    new_entities_content = f.read().replace('\n', '')

additional_entities_string = json.dumps(dh.additional_entities)

# write new and combine with old test data
with open('../datasets/all-test/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content = entities_content[0:-1] +", " + additional_entities_string[1:-1]+ ", " + new_entities_content[1:]
    f.write(str(merge_content))

# write just the new parse data to the entities json.
with open('../datasets/all-parsed/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content =  "[" + root_territory + "," + additional_entities_string[1:-1]+ ","+ r_entities +","+  new_entities_content[1:]
    f.write(str(merge_content))

relations_entities_string = json.dumps(relation_records_cleaned)
audits_entities_string = json.dumps(dh.audits)

# write relations
with open('../datasets/all-parsed/relations.json','w', encoding='utf-8') as f:
    merge_content =  relations_entities_string[0:-1] + "," + r_relations + "]"
    f.write(str(merge_content))
    #json.dump(relation_records_cleaned, f)

# write the audits json.
with open('../datasets/all-parsed/audits.json','w', encoding='utf-8') as f:
    merge_content =  audits_entities_string[0:-1] +  ","+ r_audits + "]"
    f.write(str(merge_content))
    #json.dump(dh.audits, f)

# write just the new parse data to the entities json.
with open('../datasets/all-parsed-wr/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content =  "[" + root_territory + "," + additional_entities_string[1:-1] +","+  new_entities_content[1:]
    f.write(str(merge_content))


# write relations
with open('../datasets/all-parsed-wr/relations.json','w', encoding='utf-8') as f:
    merge_content =  relations_entities_string[0:-1] + "]"
    f.write(str(merge_content))
    #json.dump(relation_records_cleaned, f)

# write the audits json.
with open('../datasets/all-parsed-wr/audits.json','w', encoding='utf-8') as f:
    merge_content =  audits_entities_string[0:-1] + "]"
    f.write(str(merge_content))


logger.info("END")

2023-01-22 10:19:38,084 INFO END


In [493]:
# simple 2 strategy

parsed_e_df = pd.DataFrame(json.load(open("../datasets/all-parsed-wr/entities.json","r"))).fillna("").set_index("id")
parsed_r_df = pd.DataFrame(json.load(open("../datasets/all-parsed-wr/relations.json","r"))).fillna("").set_index("id")
parsed_a_df = pd.DataFrame(json.load(open("../datasets/all-parsed-wr/audits.json","r"))).fillna("").set_index("id")

ddb1_e_df = pd.DataFrame(json.load(open("../datasets/experiments/21/entities.json","r"))).fillna("").set_index("id")
ddb1_r_df = pd.DataFrame(json.load(open("../datasets/experiments/21/relations.json","r"))).fillna("").set_index("id")
ddb1_a_df = pd.DataFrame(json.load(open("../datasets/experiments/21/audits.json","r"))).fillna("").set_index("id")



In [447]:
len(parsed_e_df), len(ddb1_e_df)

(29360, 27980)

In [468]:
ddb1_e_df['notes_s'] = ddb1_e_df['notes'].astype(str)

In [469]:
# robert_e_df = ddb1_e_df[(~ddb1_e_df.index.isin(parsed_e_df.index)) & (ddb1_e_df['notes'].str.contains("Import"))]
robert_e_df = ddb1_e_df[~ddb1_e_df['notes_s'].str.contains("Import")]
robert_e_df

,class,data,detail,label,language,legacyId,notes,props,references,status,isTemplate,notes_s
id,,,,,,,,,,,,
025d451c-0fce-436c-8d37-0d13ac7f90a9,C,{},supporter of heretics (i.e. Cathars),fautor hereticorum,lat,,[],"[{'bundleEnd': False, 'bundleOperator': 'a', '...",[],4,False,[]
0a2ba32a-4c7e-40e4-a684-a65e6044e3c2,E,{'logicalType': '1'},,metus huius inquisitionis,lat,,[],[],[],1,False,[]
0e8166b4-f424-4717-b29e-685899e1b3c3,A,"{'entities': {'a1': ['P', 'G'], 'a2': ['C', 'E...",bore witness (in front of sb - about st),testatus/a,lat,,[],[],[],1,False,[]
0fcf21d7-da3b-4fb4-bebe-bd5930685b12,V,{},,34v,lat,,[],[],[],1,False,[]
10d2538b-d7d7-4a4f-9e34-1f7993337dc9,V,{},,34r,lat,,[],[],[],1,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
f69921f2-5f70-4fd6-84da-16dba150656b,V,{},,34v,lat,,[],[],[],1,False,[]
f71126df-fc8f-42d0-977d-4f3414682bee,C,{},reporting,relatio,lat,,[],[],[],4,False,[]
f924fdff-4370-4b6d-a197-f2b289653117,V,{},,36r,lat,,[],[],[],1,False,[]


In [470]:
robert_r_df = ddb1_r_df[~ddb1_r_df.index.isin(parsed_r_df.index)]
robert_r_df

,entityIds,order,type,certainty
id,,,,
0002ea21-f57a-4066-95e2-3f5cc17adf45,"[6832b685-7391-4e6f-9dee-53ab048bced4, 390ac71...",10.0,A2S,
00055fff-5cbf-4e4f-a5b9-e9f8cd5d3b25,"[d08daee5-e2b4-458e-af9a-5d25942cee21, b1f631b...",10.0,REL,
00062934-e2cc-45e0-a4c2-f3efe0a790ee,"[9238a54a-8b50-4418-8a8a-5d8e1aeed77b, eabcf67...",1.0,CLA,
00079b82-f3eb-4815-b4f6-2a40ac887484,"[e7d8ae89-792a-4586-b9e2-4d83e3ea8824, adc03f9...",10.0,CLA,
00099734-9287-430a-9e50-5a98dcd4a6fc,"[469ca1dd-734b-4534-9bfc-fdaae17dc953, cbf3b28...",1.0,CLA,
...,...,...,...,...
ffea7bb7-15aa-4a61-972e-ba40fa3dea12,"[2abe4053-4018-4275-b20a-2e1021ac8d0c, 0d8e22d...",10.0,SOL,
ffef934d-3ddd-4382-8c4e-38430e3df341,"[4a4b7303-7823-43cc-8d85-546db731e3e5, 4762d66...",10.0,SOL,
fff1ddfd-e0c3-425a-856b-ec42cccacbc6,"[b2c92fdf-dc0d-4699-ac58-27c3472f52c3, 5904cba...",10.0,CLA,


In [ ]:
#  after stitching investigation


In [494]:
#  after stitching

# - R entita  s label "Reproduction of Paris, Bibliothèque nationale de France, fonds Doat, vol. 21"  musí mít guid 35a13dc0-783a-466d-9850-71c50b1f528d

with open('../datasets/all-parsed/entities.json','r', encoding='utf-8') as f:
  econtent = f.read()
  econtent = econtent.replace("ccb120c6-7687-4a93-b6d7-17c30c33fd27","35a13dc0-783a-466d-9850-71c50b1f528d")

with open('../datasets/all-parsed/entities.json','w', encoding='utf-8') as f:
  f.write(econtent)

In [495]:
# take statement content from the last downloaded DDB data
with open('../datasets/experiments/21/entities.json','r', encoding='utf-8') as f:
  # ddb1e = f.read()
  ddb1e = json.load(f)
  ddb1e_df = pd.DataFrame(ddb1e).set_index("id")

with open('../datasets/all-parsed/entities.json','r', encoding='utf-8') as f:
  # ddb1e = f.read()
  parsed_e = json.load(f)
  parsed_e_df = pd.DataFrame(parsed_e).set_index("id")


In [496]:
parsed_e_dfDD = parsed_e_df[~parsed_e_df.index.duplicated(keep='first')]

In [497]:
# reaplace all statements from ddb1e

parsed_e_dfDDnoS = parsed_e_dfDD[parsed_e_dfDD['class']!="S"]

In [498]:
ddb1s = ddb1e_df[ddb1e_df['class']=="S"]
result_e_df = pd.concat([parsed_e_dfDDnoS, ddb1s])
# parsed_e_df[parsed_e_df['class']=="S"]

In [499]:
result_e_df.reset_index().to_json('../datasets/stitched/entities.json',orient="records")

In [388]:
x = 0
for item in parsed_e_df[parsed_e_df['class']=="S"].index:
  # print(item)
  x += 1
  print(x, item)
  if item in ddb1e_df.index:
    print('*',item)

1 38e5b435-bb22-4b09-97be-7acdae65e370
* 38e5b435-bb22-4b09-97be-7acdae65e370
2 795a5729-1052-48c9-ae9a-d89a63d4c4c7
* 795a5729-1052-48c9-ae9a-d89a63d4c4c7
3 7aa92591-58cf-41c5-a4fc-d894914bedf8
* 7aa92591-58cf-41c5-a4fc-d894914bedf8
4 82bbf78f-7995-4928-9e49-de29a46d0b05
* 82bbf78f-7995-4928-9e49-de29a46d0b05
5 8338273a-151d-44a8-94ba-0ff6ac69f5db
* 8338273a-151d-44a8-94ba-0ff6ac69f5db
6 8ad01491-fff6-4762-81c2-a44316a2e32b
* 8ad01491-fff6-4762-81c2-a44316a2e32b
7 953cc66a-49e8-4386-8178-32d7aea5539f
* 953cc66a-49e8-4386-8178-32d7aea5539f
8 c3d073d2-8b30-49b1-9a46-e1260329924f
* c3d073d2-8b30-49b1-9a46-e1260329924f
9 1361864f-1682-4251-a832-c30da8216830
* 1361864f-1682-4251-a832-c30da8216830
10 1d271952-dbb0-4bf4-96e4-d20970f8dd66
* 1d271952-dbb0-4bf4-96e4-d20970f8dd66
11 29574a5a-2bd6-472e-ab43-e8507249c203
* 29574a5a-2bd6-472e-ab43-e8507249c203
12 3071ab0b-7554-43dd-b499-430e99d93fd3
* 3071ab0b-7554-43dd-b499-430e99d93fd3
13 3e51a831-2774-4262-98a4-b693aab89af8
* 3e51a831-2774-4262-

In [407]:
working = list(parsed_e_df.index)

In [409]:
len(set([x for x in working if working.count(x) > 1]))

47

In [396]:
len(working)

43

In [397]:
len(set(working))

34

In [155]:
len(dh.relation_records), len(relation_records_cleaned)

(15026, 14588)

In [19]:
dh.additional_entities


[{'class': 'E',
  'id': 'fe6753df-aeda-4d6c-8456-fabaf5ee1ab0',
  'label': 'Creation of ‘Acta Concilii Tolosani’',
  'language': '',
  'detail': '',
  'data': {'logicalType': '1'},
  'props': [],
  'notes': ['Import batch [development] 2023-01-13 19:35:45.939035',
   'special T153>:creation_event_id E0001_T153'],
  'status': '1',
  'references': [],
  'legacyId': 'E0001_T153'},
 {'class': 'E',
  'id': 'c7f31835-13a4-405c-a7aa-e630b40e14d8',
  'label': 'Creation of ‘Acta Sancti Officii Bononie ab anno 1291 usque ad annum 1310’',
  'language': '',
  'detail': '',
  'data': {'logicalType': '1'},
  'props': [],
  'notes': ['Import batch [development] 2023-01-13 19:35:45.939035',
   'special T77>:creation_event_id E0002_T77'],
  'status': '1',
  'references': [],
  'legacyId': 'E0002_T77'},
 {'class': 'V',
  'id': '09747a19-bf4b-48d1-b782-140bd1a2e8f6',
  'label': '1291',
  'language': '',
  'detail': '',
  'data': {'logicalType': '1'},
  'props': [],
  'notes': ['Import batch [development]

In [ ]:
# Log tables google sheet
# https://docs.google.com/spreadsheets/d/1UpbJckLrRYTCz7wcEYhhcc-ihJxiUENx1unCCQqE4TE/edit#gid=0


# d.open_gsheet("logtables","https://docs.google.com/spreadsheets/d/1UpbJckLrRYTCz7wcEYhhcc-ihJxiUENx1unCCQqE4TE")

# d.write_df_to_gsheet("logtables", "values", tables["values"])
# d.write_df_to_gsheet("logtables", "resources", tables["resources"])
# d.write_df_to_gsheet("logtables", "locations", tables["locations"])
# d.write_df_to_gsheet("logtables", "events", tables["events"])
# d.write_df_to_gsheet("logtables", "props", tables["props"])



In [156]:
d.open_gsheet("discouraged","https://docs.google.com/spreadsheets/d/10x8TV2OmkcGt6zHw-3B-I3BhonETa0iQV424vCB9qSQ/edit#gid=0")

20 2023-01-17 14:07:00 : Opening gsheet handle discouraged


In [157]:
discouraged_concepts = dh.tables['concepts'][tables['concepts']['status']=="discouraged"][['legacyId','label']]
discouraged_concepts_ids_list = discouraged_concepts['legacyId'].to_list()
discouraged_concepts_ids_list

['C2090',
 'C0408',
 'C0410',
 'C3069',
 'C0602',
 'C0165',
 'C0394',
 'C0541',
 'C0653',
 'C0744',
 'C0395',
 'C0417',
 'C2776',
 'C0125',
 'C0231',
 'C0611',
 'C0612',
 'C0683',
 'C0764',
 'C0784',
 'C0433',
 'C0636',
 'C0701',
 'C1271',
 'C1272',
 'C1273',
 'C0449',
 'C0455',
 'C0457',
 'C0531',
 'C0475',
 'C0672',
 'C0432',
 'C0439',
 'C0441',
 'C0673',
 'C0494',
 'C0495',
 'C0205',
 'C0206',
 'C0207',
 'C0498',
 'C0505',
 'C0518',
 'C0519',
 'C0520',
 'C2087',
 'C0203',
 'C0496',
 'C0497',
 'C0502',
 'C0503',
 'C0421',
 'C0422',
 'C0174',
 'C0176',
 'C0183',
 'C0184',
 'C0175',
 'C0177',
 'C2091',
 'C0210',
 'C0499',
 'C0500',
 'C1112',
 'C0628',
 'C0630',
 'C0180',
 'C0219',
 'C0525',
 'C0529',
 'C0627',
 'C0629',
 'C0238',
 'C0278',
 'C0515',
 'C0516',
 'C0115',
 'C0179',
 'C0240',
 'C0030',
 'C0924',
 'C0925',
 'C0926',
 'C0622',
 'C0249',
 'C0208',
 'C0028',
 'C0029',
 'C2096',
 'C0489',
 'C1158',
 'C1169',
 'C2600',
 'C2412',
 'C0023',
 'C0025',
 'C2168',
 'C0710',
 'C0443',


In [158]:
discouraged_concepts.set_index('legacyId').loc['C3324']['label']

'house of origin'

In [159]:
found = [('table','legacyId','column_with_concept','discouraged_concept_id','discouraged_concept_label')]

for t, tt in tables.items():
    #df = tables['resources']
    df = tables[t]
    #mask = np.column_stack([df[col].str.contains(r"C00", na=False) for col in df])
    mask = np.column_stack([df[col].isin(discouraged_concepts_ids_list) for col in df])
    result = df.loc[mask.any(axis=1)]
    if not result.empty:
        #found[t] = []
        print(t)
        lids = result['legacyId'].to_list()
        print(lids)
        for l in lids:
            target = df[df['legacyId'] == l]
            for c in target.columns:
                #print(target[c].iloc[0])
                if c=="legacyId":
                    continue
                if t=="concepts":
                    if target['status'].iloc[0] == "discouraged":
                        continue
                if target[c].iloc[0] in discouraged_concepts_ids_list:
                    found.append((t,l,c, target[c].iloc[0],discouraged_concepts.set_index('legacyId').loc[target[c].iloc[0]]['label']))



R0031_locations
['L0275_R0031', 'L0442_R0031']
actions
['A0182', 'A0030', 'A0104']
concepts
['C2090', 'C0408', 'C0409', 'C0410', 'C0948', 'C0949', 'C0950', 'C3069', 'C0602', 'C0165', 'C0394', 'C0541', 'C0653', 'C0744', 'C0395', 'C2003', 'C0416', 'C0417', 'C2776', 'C0125', 'C0231', 'C0611', 'C0612', 'C0683', 'C0764', 'C0784', 'C0415', 'C0433', 'C0636', 'C0806', 'C0160', 'C0647', 'C0701', 'C1230', 'C1231', 'C1271', 'C1272', 'C1273', 'C0335', 'C0194', 'C0376', 'C0449', 'C0454', 'C0455', 'C0456', 'C0457', 'C0476', 'C0530', 'C0531', 'C0474', 'C0475', 'C0672', 'C0217', 'C0432', 'C0438', 'C0439', 'C0440', 'C0441', 'C0673', 'C0860', 'C0863', 'C0864', 'C1811', 'C0822', 'C1975', 'C2774', 'C0494', 'C0495', 'C0205', 'C0206', 'C0207', 'C0498', 'C0505', 'C0518', 'C0519', 'C0520', 'C2087', 'C0203', 'C0496', 'C0497', 'C0502', 'C0503', 'C0421', 'C0422', 'C0174', 'C0176', 'C0183', 'C0184', 'C0175', 'C0177', 'C2091', 'C0210', 'C0499', 'C0500', 'C1112', 'C0628', 'C0630', 'C0180', 'C0219', 'C0525', 'C0529'

In [160]:
rdf = pd.DataFrame(found)
new_header = rdf.iloc[0] #grab the first row for the header
rdf = rdf[1:] #take the data less the header row
rdf.columns = new_header
rdf

,table,legacyId,column_with_concept,discouraged_concept_id,discouraged_concept_label
1,R0031_locations,L0275_R0031,class_id2,C0851,locus
2,R0031_locations,L0442_R0031,class_id2,C0851,locus
3,actions,A0182,related_entity_id,C0165,extra tempus gracie
4,actions,A0030,subject_semantic_id,C1465,adversary
5,actions,A0030,actant1_semantic_id,C1465,adversary
...,...,...,...,...,...
199,concepts,C2149,grammatical_number_id,C0954,singular
200,concepts,C2263,components_id,C2265,apostolici
201,concepts,C2264,plural_id,C2265,apostolici
202,concepts,C2264,grammatical_number_id,C0954,singular


In [161]:
d.write_df_to_gsheet("discouraged", "discouraged_concepts_2023", rdf)

20 2023-01-17 14:07:30 : Trying to write target dataset 'discouraged_concepts_2023' to gsheet handle: discouraged
20 2023-01-17 14:07:30 : Using existing output worksheet
20 2023-01-17 14:07:31 : Successfully written to gsheet handle: discouraged


In [ ]:
#mask = np.column_stack([df[col].str.contains(r"\^", na=False) for col in df])
#df.loc[mask.any(axis=1)]
